In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime

In [19]:
np.random.seed(7)

In [54]:
train_labels = pd.read_csv('./input/dengue_labels_train.csv', engine='python')
total_test_features = pd.read_csv('./input/dengue_features_test.csv')
train_labels = np.ravel(train_labels.drop(train_labels.columns[[0, 1, 2]], axis=1).values.astype('float32'))
sj_labels = train_labels[:935]
iq_labels = train_labels[936:]

In [55]:
dataset = sj_labels
sj_scaler = MinMaxScaler(feature_range=(0, 1))
dataset = sj_scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
sj_train, sj_test = dataset[0:train_size], dataset[train_size:len(dataset)]

dataset = iq_labels
iq_scaler = MinMaxScaler(feature_range=(0, 1))
dataset = iq_scaler.fit_transform(dataset)
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
iq_train, iq_test = dataset[0:train_size], dataset[train_size:len(dataset)]

/Users/Simran/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/Simran/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/Simran/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if yo

In [56]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [57]:
look_back = 1
sj_trainX, sj_trainY = create_dataset(sj_train, look_back)
sj_testX, sj_testY = create_dataset(sj_test, look_back)
iq_trainX, iq_trainY = create_dataset(iq_train, look_back)
iq_testX, iq_testY = create_dataset(iq_test, look_back)
# sj_x, sj_y = create_dataset(sj_final_test, look_back)
# iq_x, iq_y = create_dataset(iq_final_test, look_back)

In [58]:
# reshape input to be [samples, time steps, features]
sj_trainX = np.reshape(sj_trainX, (sj_trainX.shape[0], 1, sj_trainX.shape[1]))
sj_testX = np.reshape(sj_testX, (sj_testX.shape[0], 1, sj_testX.shape[1]))

# reshape input to be [samples, time steps, features]
iq_trainX = np.reshape(iq_trainX, (iq_trainX.shape[0], 1, iq_trainX.shape[1]))
iq_testX = np.reshape(iq_testX, (iq_testX.shape[0], 1, iq_testX.shape[1]))

# sj_x = np.reshape(sj_x, (sj_x.shape[0], 1, sj_x.shape[1]))
# iq_x = np.reshape(iq_x, (iq_x.shape[0], 1, iq_x.shape[1]))

In [60]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(sj_trainX, sj_trainY, epochs=100, batch_size=10, verbose=2)

Epoch 1/100
1s - loss: 0.0215
Epoch 2/100
0s - loss: 0.0169
Epoch 3/100
0s - loss: 0.0158
Epoch 4/100
0s - loss: 0.0150
Epoch 5/100
0s - loss: 0.0142
Epoch 6/100
0s - loss: 0.0133
Epoch 7/100
0s - loss: 0.0123
Epoch 8/100
0s - loss: 0.0112
Epoch 9/100
0s - loss: 0.0101
Epoch 10/100
0s - loss: 0.0090
Epoch 11/100
0s - loss: 0.0078
Epoch 12/100
0s - loss: 0.0067
Epoch 13/100
0s - loss: 0.0055
Epoch 14/100
0s - loss: 0.0045
Epoch 15/100
0s - loss: 0.0036
Epoch 16/100
0s - loss: 0.0028
Epoch 17/100
0s - loss: 0.0023
Epoch 18/100
0s - loss: 0.0018
Epoch 19/100
0s - loss: 0.0015
Epoch 20/100
0s - loss: 0.0014
Epoch 21/100
0s - loss: 0.0012
Epoch 22/100
0s - loss: 0.0012
Epoch 23/100
0s - loss: 0.0011
Epoch 24/100
0s - loss: 0.0011
Epoch 25/100
0s - loss: 0.0011
Epoch 26/100
0s - loss: 0.0011
Epoch 27/100
0s - loss: 0.0011
Epoch 28/100
0s - loss: 0.0011
Epoch 29/100
0s - loss: 0.0011
Epoch 30/100
0s - loss: 0.0011
Epoch 31/100
0s - loss: 0.0011
Epoch 32/100
0s - loss: 0.0011
Epoch 33/100
0s -

In [66]:
# make predictions
trainPredict = model.predict(sj_trainX)
testPredict = model.predict(sj_testX)
# invert predictions
trainPredict = sj_scaler.inverse_transform(trainPredict)
trainY = sj_scaler.inverse_transform([sj_trainY])
testPredict = sj_scaler.inverse_transform(testPredict)
testY = sj_scaler.inverse_transform([sj_testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 15.16 RMSE
Test Score: 9.37 RMSE


In [67]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(iq_trainX, iq_trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
3s - loss: 0.0100
Epoch 2/100
2s - loss: 0.0091
Epoch 3/100
2s - loss: 0.0088
Epoch 4/100
2s - loss: 0.0085
Epoch 5/100
2s - loss: 0.0082
Epoch 6/100
2s - loss: 0.0080
Epoch 7/100
2s - loss: 0.0077
Epoch 8/100
2s - loss: 0.0075
Epoch 9/100
2s - loss: 0.0072
Epoch 10/100
2s - loss: 0.0068
Epoch 11/100
2s - loss: 0.0064
Epoch 12/100
2s - loss: 0.0060
Epoch 13/100
2s - loss: 0.0057
Epoch 14/100
2s - loss: 0.0054
Epoch 15/100
2s - loss: 0.0051
Epoch 16/100
2s - loss: 0.0049
Epoch 17/100
2s - loss: 0.0047
Epoch 18/100
2s - loss: 0.0046
Epoch 19/100
2s - loss: 0.0044
Epoch 20/100
2s - loss: 0.0043
Epoch 21/100
2s - loss: 0.0043
Epoch 22/100
2s - loss: 0.0044
Epoch 23/100
2s - loss: 0.0042
Epoch 24/100
2s - loss: 0.0043
Epoch 25/100
2s - loss: 0.0042
Epoch 26/100
2s - loss: 0.0042
Epoch 27/100
2s - loss: 0.0042
Epoch 28/100
2s - loss: 0.0042
Epoch 29/100
2s - loss: 0.0042
Epoch 30/100
2s - loss: 0.0042
Epoch 31/100
2s - loss: 0.0042
Epoch 32/100
3s - loss: 0.0042
Epoch 33/100
2s -

In [68]:
# make predictions
trainPredict = model.predict(iq_trainX)
testPredict = model.predict(iq_testX)
iq_predictions = [int(i) for i in model.predict(iq_x)]
# invert predictions
trainPredict = iq_scaler.inverse_transform(trainPredict)
trainY = iq_scaler.inverse_transform([iq_trainY])
testPredict = iq_scaler.inverse_transform(testPredict)
testY = iq_scaler.inverse_transform([iq_testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

NameError: name 'iq_x' is not defined